<a href="https://colab.research.google.com/github/rahutchinson/cash_loss_risk/blob/master/Risk_for_Spending_PART1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Goal for Project
 - to wake up every moring to a percent risk of a large purchase or lots of purchases  to try to fight spending!
 - use an unsupervised neural network

# Setup

In [0]:
#@title Initial Imports
import pandas as pd
import io
import datetime as dt
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))
  
  

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:


df_amex.head()

b'Skipping line 20: expected 5 fields, saw 6\nSkipping line 45: expected 5 fields, saw 6\n'


,0,7
0,11/15/2016 Tue,1484.76
1,11/17/2016 Thu,-100.00
2,11/21/2016 Mon,13.99
3,11/22/2016 Tue,-1.40
4,11/23/2016 Wed,15.14


# Data Cleansing



Below Cell is for engineering the data in to a workable format
  - Date
  - Money Spent

In [0]:
df_amex = pd.read_csv('/content/gdrive/My Drive/Colab_Notebooks/amex.csv', header=None, usecols=[0,7])
df_amex.columns = ['date', 'cost']
df_amex.cost = pd.to_numeric(df_amex.cost)
df_amex = df_amex[df_amex.cost>0]
df_amex['date'] = df_amex.date.apply(lambda x: pd.to_datetime(x))
df_amex['moneys'] = df_amex.cost
df_amex = df_amex[['date', 'moneys']]

In [0]:
df_chase = pd.read_csv('/content/gdrive/My Drive/Colab_Notebooks/chase.CSV', error_bad_lines=False)
df_chase = df_chase[df_chase['Type'] == 'Sale']
df_chase['date'] = df_chase['Trans Date'].apply(lambda x: pd.to_datetime(x))
df_chase['moneys'] = pd.to_numeric(df_chase['Amount'])
df_chase = df_chase[['date', 'moneys']]


dfs = [df_chase, df_amex]
for df in dfs:
  display(df.head())

b'Skipping line 20: expected 5 fields, saw 6\nSkipping line 45: expected 5 fields, saw 6\n'


,date,moneys
0,2018-11-09,-26.76
1,2018-11-09,-9.06
2,2018-11-09,-7.56
3,2018-11-09,-9.54
4,2018-11-05,-8.83


,date,moneys
0,2016-11-15,1484.76
2,2016-11-21,13.99
4,2016-11-23,15.14
5,2016-11-23,6.00
7,2016-11-26,33.02


# Data Processing
Take the data and tally it up based on:
  - Day of week
  - Day of year
  - Per Day

In [0]:
dict_all = {}
for df in dfs:
  df['dow'] = df['date'].dt.weekday_name
  df['doy'] = df['date'].apply(lambda x: x.timetuple().tm_yday)
  
  

   
metric = [0,0]

dow_count = {'Sunday': [0,0], 'Monday': [0,0], 'Tuesday': [0,0], 
             'Wednesday': [0,0], 'Thursday': [0,0], 'Friday': [0,0],
             'Saturday': [0,0]}

for dow in dow_count:
  for df in dfs:
    dow_count[dow][0] += df[df['dow'] == dow].count()['dow']
    dow_count[dow][1] += df[df['dow'] == dow].sum()['moneys']
doy_counts = {}
for i in range(0,366):
  doy_counts[i] = [0,0]
for day in doy_counts:
  for df in dfs:
    doy_counts[day][0] += df[df['doy'] == day].count()['doy']
    doy_counts[day][1] += df[df['doy'] == day].sum()['moneys']

count_dow=[]
dollas_dow=[]
for day in dow_count:
  count_dow.append(dow_count.get(day)[0])
  dollas_dow.append(dow_count.get(day)[1])    
    

count_doy=[]
dollas_doy=[]
for day in doy_counts:
  count_doy.append(doy_counts.get(day)[0])
  dollas_doy.append(doy_counts.get(day)[1])
  
  
overall = pd.concat(dfs)

# Data Visualized

In [0]:
#@title Day of Week
counts = go.Scatter(
    x = ['sunday','monday','tue','wed','thur','fri','sat'],
    y = count_dow,
    name="dates"
)

money = go.Scatter(
    x = ['sunday','monday','tue','wed','thur','fri','sat'],
    y = dollas_dow,
    name="moneys",
    yaxis='y2'
)
data = [counts, money]
configure_plotly_browser_state()
layout = go.Layout(
    title='When I Buy and How much',
    yaxis=dict(
        title='Amount of Purchases'
    ),
    yaxis2=dict(
        title='Amount of money spent',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [0]:
#@title Day of Year
counts = go.Scatter(
    x = list(doy_counts.keys()),
    y = count_doy,
    name="dates"
)

money = go.Scatter(
    x = list(doy_counts.keys()),
    y = dollas_doy,
    name="moneys",
    yaxis='y2'
)
data = [counts, money]
configure_plotly_browser_state()
layout = go.Layout(
    title='When I Buy and How much',
    yaxis=dict(
        title='Amount of Purchases'
    ),
    yaxis2=dict(
        title='Amount of money spent',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [0]:
#@title Overall
counts = go.Scatter(
    x = overall['date'],
    y = overall['moneys'],
    name="dates",
    mode = 'markers'
)

data = [counts]
configure_plotly_browser_state()
layout = go.Layout(
    title='Overall',
    yaxis=dict(
        title='money spent'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

# Training


In [0]:
from fbprophet import Prophet